In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd
from google.colab import files
import io

def processar_pagamentos_colab():
    print("Sistema de Conferência de Pagamentos por CPF")

    # 1. Upload dos arquivos
    uploaded = files.upload()

    nome_arq_banco = ""
    nome_arq_empresa = ""

    for nome_arquivo in uploaded.keys():
        if "banco" in nome_arquivo.lower():
            nome_arq_banco = nome_arquivo
        elif "empresa" in nome_arquivo.lower():
            nome_arq_empresa = nome_arquivo

    if not nome_arq_banco or not nome_arq_empresa:
        print("Erro: Arquivos não identificados.")
        return

    # 2. Data de competência
    data_competencia = input("Informe a Data de Competência: ")

    try:
        print("Processando...")

        # 3. Leitura e preparação da base do banco
        df_banco = pd.read_excel(io.BytesIO(uploaded[nome_arq_banco]))

        dicionario_banco = dict(zip(
            df_banco.iloc[:, 2].astype(str).str.strip(),  # CPF
            df_banco.iloc[:, 3]                           # Matrícula
        ))

        # 4. Leitura da base da empresa
        df_empresa = pd.read_excel(io.BytesIO(uploaded[nome_arq_empresa]))

        lista_final = []

        # 5. Validação por CPF
        for _, linha in df_empresa.iterrows():

            nome = str(linha.iloc[1])
            cpf = str(linha.iloc[3]).strip()
            valor = linha.iloc[7]

            matricula = dicionario_banco.get(cpf, "Não Cadastrado no Banco")

            lista_final.append({
                "Nome do Beneficiário": nome,
                "CPF": cpf,
                "Valor": valor,
                "Data de Competência": data_competencia,
                "Matrícula": matricula
            })

        # 6. Geração do arquivo final
        if lista_final:
            nome_saida = "Planilha_Pagamento_Final_Por_CPF.xlsx"
            pd.DataFrame(lista_final).to_excel(nome_saida, index=False)
            print(f"Sucesso! {len(lista_final)} registros processados.")
            files.download(nome_saida)
        else:
            print("Nenhum registro processado.")

    except Exception as e:
        print(f"Erro: {e}")

if __name__ == "__main__":
    processar_pagamentos_colab()